In [253]:
import pandas as pd
import plotly.express as px
import numpy as np
import json
import json5

In [254]:
import usb
usb.core.find(find_all=True)

<generator object find.<locals>.device_iter at 0x124b415f0>

In [281]:
#pd.read_json("./log/pymmw_2023-01-29_23-54-25.log", lines=True)
i = 0
with open("./log/pymmw_2023-01-29_23-54-25.log", 'r') as f:
    l = []
    for line in f:
        try:
            l.append(json5.loads(line))
        except:
            pass
        if i == 5:
            break
        i+= 1
l
t = [i['ts'] for i in l]

In [309]:
def parse_dframe(d: dict, IDX: int = 0):
    ts = pd.Timestamp.now()
    df = pd.DataFrame.from_dict(d['detected_points'], orient='index').reset_index(drop=True).assign(frame=IDX, ts=ts)
    return {
        'ts':   ts,
        'frame': IDX,
        'xyzv': df[['x','y','z','v']].to_numpy().tolist()
    }
def parse_df(d: dict, idx: int = 0):
    ts = pd.to_datetime(d['ts'], unit='ms')
    df = pd.DataFrame.from_dict(d['dataFrame']['detected_points'], orient='index').reset_index(drop=True).assign(frame=idx, ts=ts)
    return df

In [301]:
_ = parse_dframe(l[0]['dataFrame'])
_.pop('frame',None)
_
pd.Series(pd.to_datetime(t, unit='ms')).dtypes
pd.to_datetime(1678830096007, unit='ms')

Timestamp('2023-03-14 21:41:36.007000')

In [334]:
def read_log(file_str) -> pd.DataFrame:
    with open(file_str, 'r') as f:
        l, i = [], 0
        for line in f:
            try:
                l.append(json5.loads(line)|{'frame':i})
            except:
                pass
            i+= 1
    df = pd.DataFrame()
    for row in l:
        df = pd.concat([df, parse_df(row, row['frame'])])
    return df

In [335]:
df = read_log("./log/pymmw_2023-01-29_23-54-25.log")

In [377]:
S = df[df['v'] == df['v'].max()].iloc[0]
tss = df['ts']
tsss = (df['ts'].astype(int) / 10**6).astype(int)
tss_ = S['ts']
tsss_ = int(S['ts'].to_datetime64().astype(int) / 10**6)

In [378]:
%timeit tss_ in tss.values

6.68 µs ± 31.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [379]:
%timeit tsss_ in tsss.values

2.39 µs ± 30.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [381]:
tss_  in tss.values

True

In [388]:
%timeit int(pd.Timestamp.now().to_datetime64().astype(int) / 10**6)

2.08 µs ± 33 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [398]:
int(pd.Timestamp.now().to_datetime64().astype(int) / 10**6)

1679365421230

In [402]:
S[['x','y','z','v']].tolist()

[-0.07325440645217896, 1.1697790622711182, 0.0, 5.461647987365723]

In [410]:
pd.Series(S['ts'])

0   2023-01-29 23:56:09.945
dtype: datetime64[ns]

In [413]:
%timeit x = {'ts':S['ts'],'frame':i,'xyzv':S[['x','y','z','v']].to_numpy().tolist()}

127 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [415]:
%timeit x = dict(ts=S['ts'],frame=i,xyzv=df[['x','y','z','v']].to_numpy().tolist())

449 µs ± 9.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [417]:
len(S)

6

In [419]:
print(S.to_dict())

{'v': 5.461647987365723, 'x': -0.07325440645217896, 'y': 1.1697790622711182, 'z': 0.0, 'frame': 1602, 'ts': Timestamp('2023-01-29 23:56:09.945000')}


In [426]:
dff = df.loc[df['z'] > 10]
dff.assign(outliers=0, model=0)

,v,x,y,z,frame,ts,outliers,model


In [ ]:
S = df[df['v'] == df['v'].max()].iloc[0]
S['ts'] in tss.values

In [436]:
pd.concat([tsss, pd.Series(tsss_)])

0    1675036467759
1    1675036467759
2    1675036467759
0    1675036467809
1    1675036467809
         ...      
0    1675036671881
0    1675036671931
0    1675036671981
0    1675036672030
0    1675036569945
Length: 4476, dtype: int64

In [440]:
_df_ = pd.DataFrame(columns=['fds','fdsd','fddd'])
_df_.loc[_df_['fds'] == _df_['fds'].max()]

,fds,fdsd,fddd


In [466]:
dff = df.loc[df['v'] == df['v'].max()]
#dff[['x','y','z','v']].iloc[0].tolist()
dff[['x','y','z','v']].iloc[0].tolist()
df.copy(deep=True).assign(w=90)
dff[['x','y','z','v']].iloc[0].tolist()

[-0.07325440645217896, 1.1697790622711182, 0.0, 5.461647987365723]

In [324]:
df = pd.DataFrame(columns=['x','ts'])
pd.concat([df.loc[df['ts'] >= 0], pd.DataFrame({'x': [[0,432]], 'ts':[[5435,432]]})])
pd.DataFrame({'x': [0,543,543], 'ts':[5435,543,543]})

,x,ts
0,0,5435
1,543,543
2,543,543


In [473]:
dff.drop(4)['v'].max()

nan

In [468]:
HDBSCAN_PARAMS = {
            # REF: https://hdbscan.readthedocs.io/en/latest/api.html#hdbscan.hdbscan_.HDBSCAN
            'algorithm': 'best',
            'min_cluster_size': 3,
            'metric': 'euclidean',
            'cluster_selection_method': 'eom', # TODO: test 'leaf'
            'alpha': 1.0,
        }
HDBSCAN_PARAMS

{'algorithm': 'best',
 'min_cluster_size': 3,
 'metric': 'euclidean',
 'cluster_selection_method': 'eom',
 'alpha': 1.0}

In [325]:
PARAMS = {
            'min_cluster_size': 3,
            'algorithm':        'best',
            'alpha':            1.0,
            'metric':           'euclidean',
        }
PARAMS

{'min_cluster_size': 3,
 'algorithm': 'best',
 'alpha': 1.0,
 'metric': 'euclidean'}

,x,ts


In [330]:
_t = pd.Series()
for i in t:
    _t = pd.concat([_t, pd.Series(pd.to_datetime(i,unit='ms'))], ignore_index=True)
_t

/var/folders/8j/pyhkkq5n0fsdymsywt9t5mh00000gn/T/com.apple.shortcuts.mac-helper/ipykernel_76891/1092995800.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _t = pd.Series()


0   2023-01-29 23:54:27.759
1   2023-01-29 23:54:27.809
2   2023-01-29 23:54:27.859
3   2023-01-29 23:54:27.909
4   2023-01-29 23:54:27.959
5   2023-01-29 23:54:28.009
dtype: datetime64[ns]

In [329]:
_t = pd.Series()
for i in t:
    _t += pd.to_datetime(i,unit='ms')
_t

/var/folders/8j/pyhkkq5n0fsdymsywt9t5mh00000gn/T/com.apple.shortcuts.mac-helper/ipykernel_76891/1708449439.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _t = pd.Series()


TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

## benchmarking

In [187]:
%%timeit
df, ts = pd.DataFrame(), pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
for p in d['detected_points'].values():
    df = pd.concat([
        df,
        pd.Series(p.values(), index=['x','y','z','v']).to_frame().transpose()
        #pd.Series(p.values(), index=['x','y','z','v']).to_frame().transpose()
        #pd.DataFrame({'x': p['x'], 'y': p['y'], 'z':p['z'], 'v': p['v']})
    ])
df = df.assign(frame=0, ts=ts)

772 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [188]:
%%timeit
df, ts = pd.DataFrame(), pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
for p in d['detected_points'].values():
    df = pd.concat([
        df,
        pd.Series(p.values(), index=['x','y','z','v']).to_frame().T
        #pd.Series(p.values(), index=['x','y','z','v']).to_frame().transpose()
        #pd.DataFrame({'x': p['x'], 'y': p['y'], 'z':p['z'], 'v': p['v']})
    ])
df = df.assign(frame=0, ts=ts)

768 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [235]:
pd.DataFrame.from_dict(d['detected_points'], orient='index')

,v,x,y,z
"0,0",0.0,2.316671,2.240985,0.0
"1,1",0.0,-2.380768,5.354967,0.0
"2,2",0.0,-3.516212,6.090257,0.0


In [207]:
%%timeit
df, ts = pd.DataFrame(), pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
for p in d['detected_points'].values():
    df = pd.concat([
        df,
        pd.DataFrame({'x': [p['x']], 'y': [p['y']], 'z':[p['z']], 'v': [p['v']]})
    ], ignore_index=True)
df = df.assign(frame=0, ts=ts)

610 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [227]:
%%timeit
df, ts = pd.DataFrame(), pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
for p in d['detected_points'].values():
    df = pd.concat([
        df,
        pd.DataFrame().from_dict(p, orient='index').T
    ], ignore_index=True)
df = df.assign(frame=0, ts=ts)
df

870 µs ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [245]:
%%timeit
ts = pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
df = pd.DataFrame.from_dict(d['detected_points'], orient='index').reset_index(drop=True).assign(frame=0, ts=ts)
df

294 µs ± 4.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [247]:
d

{'header': {'version': 3060000,
  'length': 128,
  'platform': 106843,
  'number': 1,
  'time': 441344463,
  'objects': 3,
  'blocks': 2,
  'subframe': 0},
 'detected_points': {'0,0': {'v': 0.0,
   'x': 2.3166706562042236,
   'y': 2.24098539352417,
   'z': 0.0},
  '1,1': {'v': 0.0, 'x': -2.380768299102783, 'y': 5.35496711730957, 'z': 0.0},
  '2,2': {'v': 0.0, 'x': -3.51621150970459, 'y': 6.090257167816162, 'z': 0.0}}}

In [217]:
pd.DataFrame.from_dict(data=p, orient='tight')

KeyError: 'data'

In [205]:
%timeit pd.DataFrame.from_dict(p, orient='index').T

92.4 µs ± 3.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [209]:
%%timeit
df, ts = pd.DataFrame(), pd.Timestamp.now()
idx = 0
d = l[0]['dataFrame']
for p in d['detected_points'].values():
    df = pd.concat([
        df,
        pd.DataFrame.from_dict(p, orient='index').T
        #pd.DataFrame(p.values(), columns=['x','y','z','v'])
    ], ignore_index=True)
df = df.assign(frame=0, ts=ts)

698 µs ± 9.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [186]:
{
    'ts':   int(ts.to_datetime64().astype(int) / 10**6),  # milliseconds
    'xyzv': df[['x','y','z','v']].to_numpy().tolist()
}

{'ts': 1679357037769,
 'xyzv': [[0.0, 2.3166706562042236, 2.24098539352417, 0.0],
  [0.0, -2.380768299102783, 5.35496711730957, 0.0],
  [0.0, -3.51621150970459, 6.090257167816162, 0.0]]}

In [185]:
%timeit int(ts.to_datetime64().astype(int) / 10**6)

818 ns ± 6.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:

def clean_ugly_json_file(file_in: str, file_out: TextIOWrapper) -> None:
    """
        Open shitty horrible trash ill-formatted log files and make them something less terrible.
    """
    
    json_in = []
    new_data = []

    with open(file_in,'r') as f:
        for line in f:
            try:
                json_in.append(json5.loads(line))
            except:
                pass

    idx = 0
    out_json = ""

    for i in json_in:
        
        lvl, arr = {}, []
        lvl['idx'] = int(idx)
        lvl['ts'] = int(i['ts'])

        for pt in i['dataFrame']['detected_points'].values():
            interim_arr = [pt['x'], pt['y'], pt['z'], pt['v']]
            arr.append(interim_arr)
            # np.array([pt['x'], pt['y'], pt['z'], pt['v']])
            #arr = np.append(arr, interim_arr, )

        lvl['xyzv'] = arr
        #new_data.append(lvl)
        out_json = out_json + json5.dumps(lvl, quote_keys=True) + ",\n"
        idx += 1
    
    #out_json = json5.dumps(new_data, indent=4, trailing_commas=True)[1:][:-1]
    file_out.write(out_json)